# Airflow and AWS

## Overview
1. Tạo IAM user -> Cấp policy Admin, Redshift, S3 -> Tạo Access key
2. Trong Airflow UI, tạo connection với AWS (dùng Access key đã tạo)
3. Tạo S3 bucket và copy data vào từ S3 bucket của Udacity
4. Trong DAG, dùng S3Hook để kết nối với S3 bucket và log files
5. Tạo IAM role, cấp policy Redshift, S3
6. Tạo Redshift Serverless (dùng IAM role vừa tạo)
7. Trong Airflow UI, tạo connection với Redshift
8. Trong DAG, dùng PostgresHook để kết nối với Redshift -> Tạo tables
9. Vào Redshift workgroup để query data

1. Tạo một IAM user trên AWS với 3 policies: `AdministratorAccess`, `AmazonRedshiftFullAccess`, `AmazonS3FullAccess`. Tạo Access key và lưu lại.
2. Trong Airflow, vào **Admin** -> **Connections**. Tạo connection với AWS như ở dưới. \
   (Nhớ install package `apache-airflow-providers-amazon` bản mới nhất và đặt `test_connection = Enabled` trong file `airflow.cfg`)

   <img src="images/aws1.png" width=1500>

3. Tạo S3 bucket và copy data vào
   ```bash
   aws s3 mb s3://nd027-hieu
   aws s3 cp s3://udacity-dend/data-pipelines/ ~/data-pipelines/ --recursive
   aws s3 cp ~/data-pipelines/ s3://nd027-hieu/data-pipelines/ --recursive
   aws s3 ls s3://nd027-hieu/data-pipelines/
   ```

4. Trong Airflow, vào **Admin** -> **Variables**. Tạo các variables như ở dưới:

   <img src="images/aws2.png" width=1500>

5. Xem hai files `ex1_sql_statements.py` và `ex2_connections_hooks.py`. Nhớ sửa lại tên S3 bucket. Chạy file `ex2_connections_hooks.py` trên Airflow, sẽ log các files ở trong S3 bucket trên. Chạy xong vào xem log file sẽ thấy.

6. Tạo Redshift Role trên AWS
   ```bash
   aws iam create-role --role-name my-redshift-service-role --assume-role-policy-document '{
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "redshift.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }'
   ```

7. Cấp full access to S3 cho role vừa tạo
   ```bash
   aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess --role-name my-redshift-service-role
   ```

8. Vào **Redshift** -> **Redshift Serverless** \
   Chọn **Customize settings**, điền admin và password. \
   Trong **Associated IAM roles**, Chọn **Associate IAM roles**, chọn `my-redshift-service-role`.
   Tick **Turn on enhanced VPC routing**, còn lại để nguyên -> **Save configuration** \
   Vào Workgroup configuration để kiểm tra.

   <img src="images/aws3.png" width=1500>

9. Ấn vào Workgroup vừa tạo. Trong phần **Network and security**, chọn **Edit**.

   <img src="images/aws4.png" width=1500>

10. Tick **Turn on Publicly accessible** -> **Save changes**

    <img src="images/aws5.png" width=700>

11. Ấn vào link ớ dưới **VPC security group**, sẽ đc đưa tới Security Groups trong EC2. \
    Ấn vào **Security group**, thêm một **Inboud rule**.

    <img src="images/aws6.png" width=1500>

12. Tạo connection với Redshift trong Airflow

    <img src="images/aws7.png" width=1500>

13. Xem file `ex3_s3_to_redshift.py`. Chạy file này trên Airflow, sẽ tạo một table `trips` trong Redshift -> Load data vào đó từ file `divvy_trips_2018.csv` trong S3 -> Tạo table `station_traffic`. \
    Lưu ý: Để import trực tiếp code từ một file khác (ở đây có `import sql_statement`), ko đc để các files trong subfolder ở folder `home/airflow/dags`. Nếu muốn để file trong subfolder, phải đặt tên subfolder ko có space, VD `c5l3`, và dùng `from c5l3 import sql_statements`.
14. Vào worksgroup trên Redshift, ấn vào **Query data**, sẽ đc đưa tới **Query editor** trong Redshift.

    <img src="images/aws8.png" width=1500>

15. Ấn vào dấu $>$ bên cạnh tên workgroup để kết nối tới DB.

    <img src="images/aws9.png" width=700>

16. Để nguyên, ấn **Create connection**.

    <img src="images/aws10.png" width=400>

17. Vào xem các tables đã tạo. Ấn chuột phải vào table "station_traffic" -> **Select table** -> **Run**

    <img src="images/aws11.png" width=1500>
